
---

# SQL FOR AI/ML & DATA SCIENCE — COMPLETE NOTES

---

## 1. WHAT SQL IS (INTERVIEW CONTEXT)

**SQL (Structured Query Language)** is used to:

* Extract data for **EDA**
* Build **training datasets**
* Perform **aggregations, joins, window analytics**
* Validate ML outputs
* Work with **data warehouses** (BigQuery, Redshift, Snowflake)

👉 In interviews, SQL = **data reasoning + correctness + efficiency**

---

## 2. DATABASE FUNDAMENTALS

### Types of Databases

* **Relational (SQL)**: MySQL, PostgreSQL, SQL Server
* **Analytical / OLAP**: BigQuery, Redshift, Snowflake
* **Row vs Column Stores**

  * Row-based → OLTP
  * Column-based → Analytics (faster aggregates)

---

## 3. TABLES, ROWS, COLUMNS

```sql
CREATE TABLE users (
    user_id INT PRIMARY KEY,
    name VARCHAR(50),
    age INT,
    city VARCHAR(50),
    signup_date DATE
);
```

### Data Types (Important)

* `INT`, `BIGINT`
* `FLOAT`, `DECIMAL` (⚠ ML datasets need precision)
* `VARCHAR`, `TEXT`
* `DATE`, `TIMESTAMP`
* `BOOLEAN`

---

## 4. BASIC SQL COMMANDS (MUST KNOW COLD)

### SELECT

```sql
SELECT name, age FROM users;
```

### WHERE

```sql
SELECT * FROM users WHERE age > 25;
```

### AND / OR / NOT

```sql
WHERE age > 25 AND city = 'Delhi'
```

### DISTINCT

```sql
SELECT DISTINCT city FROM users;
```

### ORDER BY

```sql
ORDER BY signup_date DESC;
```

### LIMIT / OFFSET

```sql
LIMIT 10 OFFSET 20;
```

---

## 5. AGGREGATION FUNCTIONS (VERY IMPORTANT)

### Core Aggregates

* `COUNT(*)`
* `SUM()`
* `AVG()`
* `MIN()`, `MAX()`

```sql
SELECT city, COUNT(*) 
FROM users
GROUP BY city;
```

---

## 6. GROUP BY & HAVING (INTERVIEW FAVORITE)

### GROUP BY

```sql
SELECT city, AVG(age)
FROM users
GROUP BY city;
```

### HAVING (filters groups, not rows)

```sql
SELECT city, COUNT(*)
FROM users
GROUP BY city
HAVING COUNT(*) > 10;
```

**INTERVIEW TRAP:**
❌ `WHERE COUNT(*) > 10` → INVALID
✅ `HAVING COUNT(*) > 10`

---

## 7. JOINS (CRITICAL FOR DATA SCIENCE)

### INNER JOIN

```sql
SELECT u.name, o.amount
FROM users u
INNER JOIN orders o
ON u.user_id = o.user_id;
```

### LEFT JOIN (MOST USED)

```sql
SELECT u.name, o.amount
FROM users u
LEFT JOIN orders o
ON u.user_id = o.user_id;
```

### RIGHT JOIN

### FULL OUTER JOIN

---

### Join Interview Questions

* Difference between **INNER vs LEFT JOIN**
* When LEFT JOIN produces NULLs
* How joins affect row counts
* Join vs Subquery performance

---

## 8. SUBQUERIES

### Scalar Subquery

```sql
SELECT *
FROM users
WHERE age > (SELECT AVG(age) FROM users);
```

### IN Subquery

```sql
SELECT *
FROM users
WHERE user_id IN (SELECT user_id FROM orders);
```

### Correlated Subquery

```sql
SELECT *
FROM users u
WHERE EXISTS (
    SELECT 1 FROM orders o WHERE o.user_id = u.user_id
);
```

**INTERVIEW TIP:**
JOIN > Subquery (usually faster & readable)

---

## 9. CASE WHEN (VERY IMPORTANT)

```sql
SELECT name,
       CASE
           WHEN age < 18 THEN 'Minor'
           WHEN age BETWEEN 18 AND 60 THEN 'Adult'
           ELSE 'Senior'
       END AS age_group
FROM users;
```

Used heavily in:

* Feature engineering
* Bucketing
* Label creation

---

## 10. NULL HANDLING (COMMON TRAPS)

### NULL ≠ 0 ≠ Empty String

```sql
WHERE column IS NULL
WHERE column IS NOT NULL
```

### COALESCE

```sql
COALESCE(salary, 0)
```

### IFNULL / NVL

```sql
IFNULL(salary, 0)
```

---

## 11. STRING FUNCTIONS

* `LOWER()`, `UPPER()`
* `TRIM()`
* `LENGTH()`
* `SUBSTRING()`
* `CONCAT()`

```sql
SELECT CONCAT(first_name, ' ', last_name) FROM users;
```

---

## 12. DATE & TIME FUNCTIONS (VERY IMPORTANT)

```sql
CURRENT_DATE
CURRENT_TIMESTAMP
```

```sql
DATEDIFF(end_date, start_date)
```

```sql
DATE_ADD(date, INTERVAL 7 DAY)
```

```sql
EXTRACT(YEAR FROM signup_date)
```

---

## 13. WINDOW FUNCTIONS (TOP AI/DS INTERVIEW TOPIC)

### Syntax

```sql
function() OVER (PARTITION BY column ORDER BY column)
```

---

### ROW_NUMBER

```sql
SELECT *,
ROW_NUMBER() OVER (PARTITION BY city ORDER BY signup_date) AS rn
FROM users;
```

---

### RANK vs DENSE_RANK

```sql
RANK()
DENSE_RANK()
```

---

### LAG / LEAD

```sql
SELECT user_id,
       amount,
       LAG(amount) OVER (ORDER BY date) AS prev_amount
FROM transactions;
```

Used for:

* Time-series
* Feature generation
* Trend analysis

---

### RUNNING TOTAL

```sql
SUM(amount) OVER (ORDER BY date)
```

---

## 14. CTEs (WITH CLAUSE)

```sql
WITH avg_age AS (
    SELECT AVG(age) AS avg_val FROM users
)
SELECT *
FROM users
WHERE age > (SELECT avg_val FROM avg_age);
```

**Why interviewers love CTEs:**

* Readability
* Modular logic
* Debuggable

---

## 15. SET OPERATIONS

* `UNION`
* `UNION ALL`
* `INTERSECT`
* `EXCEPT`

```sql
SELECT user_id FROM table1
UNION
SELECT user_id FROM table2;
```

---

## 16. INDEXES (PERFORMANCE QUESTIONS)

### What is an Index?

* Data structure to speed up search
* Trades **write speed for read speed**

```sql
CREATE INDEX idx_user_city ON users(city);
```

### Interview Questions

* When index helps / hurts
* Index on low vs high cardinality
* Composite index

---

## 17. NORMALIZATION

### 1NF → Atomic values

### 2NF → No partial dependency

### 3NF → No transitive dependency

**Data Science Note:**
Warehouses often use **denormalization** for performance.

---

## 18. TRANSACTIONS & ACID

* **Atomicity**
* **Consistency**
* **Isolation**
* **Durability**

```sql
BEGIN;
UPDATE accounts SET balance = balance - 100;
UPDATE accounts SET balance = balance + 100;
COMMIT;
```

---

## 19. SQL FOR FEATURE ENGINEERING (VERY IMPORTANT)

Examples:

* User activity count
* Rolling averages
* Time since last event
* Category encoding using CASE

```sql
COUNT(*) OVER (PARTITION BY user_id)
```

---

## 20. SQL FOR DATA SCIENCE INTERVIEWS (REAL QUESTIONS)

### 1. Find second highest salary

```sql
SELECT MAX(salary)
FROM employees
WHERE salary < (SELECT MAX(salary) FROM employees);
```

---

### 2. Find duplicate rows

```sql
SELECT email, COUNT(*)
FROM users
GROUP BY email
HAVING COUNT(*) > 1;
```

---

### 3. Top N per group

```sql
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) rn
    FROM employees
) t
WHERE rn <= 3;
```

---

### 4. Rolling 7-day average

```sql
AVG(sales) OVER (
    ORDER BY date
    ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
)
```

---

## 21. SQL OPTIMIZATION (INTERVIEW GOLD)

* Avoid `SELECT *`
* Use `WHERE` before `GROUP BY`
* Prefer `JOIN` over subqueries
* Index join columns
* Use `EXPLAIN`

---

## 22. SQL IN REAL AI/ML PIPELINES

* Data extraction → Pandas
* Feature generation → SQL
* Validation → SQL checks
* Monitoring drift → SQL aggregates

---

## 23. COMMON SQL INTERVIEW MISTAKES

❌ Forgetting GROUP BY columns
❌ Confusing WHERE vs HAVING
❌ Ignoring NULLs
❌ Wrong join type
❌ Not handling duplicates

---

## 24. WHAT TO MASTER FOR AIML / DATA SCIENCE ROLES

**Absolute Must:**

* JOINs
* GROUP BY + HAVING
* Window Functions
* CTEs
* Date operations
* NULL handling

**Advanced Edge:**

* LAG/LEAD
* Rolling metrics
* Performance reasoning

---


